In [18]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [19]:
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(
    groq_api_key = groq_api_key,
    model_name = "openai/gpt-oss-120b",
)

In [20]:
# create a state

class BlogState(TypedDict):
    topic:str
    outline:str
    content:str
    score:int

In [21]:
def create_outline(state : BlogState) -> BlogState:

    topic = state['topic']
    prompt = f'Generate the outline of the topic {topic}'

    answer = llm.invoke(prompt).content

    state['outline']=answer

    return state

In [22]:
def create_blog(state : BlogState) -> BlogState:

    topic = state['topic']
    outline = state['outline']

    prompt = f'write a detailed blog of the topic {topic} using the following outline {outline}'
    answer = llm.invoke(prompt).content

    state['content'] = answer
    
    return state 


In [23]:
def evaluate_blog(state : BlogState) -> BlogState:

    outline = state['outline']
    content = state['content']

    prompt = f'Give me a score between 1 to 5 based on the outline {outline} and its content {content}'
    answer = llm.invoke(prompt).content

    state['score'] = answer
    return state

In [24]:
graph = StateGraph(BlogState)

graph.add_node("create_outline",create_outline)
graph.add_node("create_blog",create_blog)
graph.add_node("evaluate_blog",evaluate_blog)

graph.add_edge(START,"create_outline")
graph.add_edge("create_outline","create_blog")
graph.add_edge("create_blog","evaluate_blog")
graph.add_edge("evaluate_blog",END)

workflow = graph.compile()


In [25]:
initial_state={
    "topic":"Rise of AI in India",
}

final_state = workflow.invoke(initial_state)

print(final_state)

{'topic': 'Rise of AI in India', 'outline': '**Outline: “Rise of AI in India”**\n\n---\n\n### 1. Introduction  \n   - 1.1 Definition of Artificial Intelligence (AI)  \n   - 1.2 Why AI matters for a rapidly developing economy  \n   - 1.3 Overview of India’s current AI landscape (size, growth rate, global ranking)  \n\n### 2. Historical Context  \n   - 2.1 Early research (1970s‑1990s) – IITs, ISI, and DRDO labs  \n   - 2‑1.1 Pioneering projects (e.g., expert systems for agriculture, early speech recognition)  \n   - 2.2 Turn of the millennium – emergence of IT services and data‑centric outsourcing  \n   - 2.3 Post‑2010 acceleration – availability of big data, cloud, and GPUs  \n\n### 3. Government Vision & Policy Framework  \n   - 3.1 National Strategy for Artificial Intelligence (NITI Aayog, 2018)  \n   - 3.2 “Digital India” & “Make in India” as enablers  \n   - 3.3 Key policy documents & initiatives  \n        - AI for All – sector‑specific roadmaps (health, agriculture, education, sma

In [26]:
print(final_state['score'])

**Score: 4 / 5**

**Why:**  
- The outline is **comprehensive** (covers history, policy, ecosystem, sector‑specific adoption, infrastructure, challenges, opportunities, case studies, and actionable recommendations).  
- It’s **well‑structured** with clear hierarchy, tables, and bullet points that make it easy to expand into a full report or article.  
- It includes **specific examples, data points, and references** (e.g., startup names, policy documents, impact metrics), which adds depth and credibility.  

**Room for improvement:**  
- Some sections could benefit from tighter focus (e.g., merging overlapping subsections in the historical context).  
- Adding more recent quantitative benchmarks (2024‑2025) and citation sources would strengthen the evidence base.  

Overall, the outline is strong and ready to be developed into a detailed piece on the rise of AI in India.
